# Curso NLP + Transformers

<img src="https://yaelmanuel.com/wp-content/uploads/2021/12/platzi-banner-logo-matematicas.png" width="500px">

---

# Análisis de Reseñas de Amazon 📦🔍

### 1) Cargar el dataset 🤓

Descomprimir archivo rar

In [ ]:
!unrar x "/content/reviews_dataframe_completo.rar"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/reviews_dataframe_completo.rar

Extracting  reviews_dataframe_completo.csv                                29% 59% 89%100%  OK 
All OK


In [ ]:
import pandas as pd

In [ ]:
csv_path = "/content/reviews_dataframe_completo.csv"
data = pd.read_csv(csv_path)

In [ ]:
data.head(3)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0491108,product_es_0296024,reviewer_es_0999081,1,Nada bueno se me fue ka pantalla en menos de 8...,television Nevir,es,electronics
1,es_0869872,product_es_0922286,reviewer_es_0216771,1,"Horrible, nos tuvimos que comprar otro porque ...",Dinero tirado a la basura con esta compra,es,electronics
2,es_0811721,product_es_0474543,reviewer_es_0929213,1,Te obligan a comprar dos unidades y te llega s...,solo llega una unidad cuando te obligan a comp...,es,drugstore


### 2) Preparación de la data 👌

#### 2.1) Instalamos las dependencias 🙌

In [ ]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

import evaluate

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from datasets import Dataset, DatasetDict

#### 2.2) Acondicionar columnas 🔍

Separar los dataset en train, test y validation.

- df_train_es (70% del dataset original)
- df_test_es (20% del dataset original)
- df_val_es (10% del dataset original)

In [ ]:
from sklearn.model_selection import train_test_split

# Paso 1: Dividir en entrenamiento (70%) y el resto (30%)
df_train_es, df_temp_es = train_test_split(data, test_size=0.3, random_state=42)  # Usa un random_state para reproducibilidad

# Paso 2: Dividir el resto (30%) en prueba (20%) y validación (10%)
df_test_es, df_val_es = train_test_split(df_temp_es, test_size=2/3, random_state=42)  # 20/30 = 2/3

Vamos a asignar una etiqueta basado en la cantidad de estrellas:
- Si el número de estrellas es mayor o igual a 3, le asignamos una buena calificación (valor 1).
- Caso contrario es una mala calificación (valor 0).

In [ ]:
df_train_es['labels'] = df_train_es['stars'].apply(lambda x: 1 if x >= 3 else 0)
df_test_es['labels'] = df_test_es['stars'].apply(lambda x: 1 if x >= 3 else 0)
df_val_es['labels'] = df_val_es['stars'].apply(lambda x: 1 if x >= 3 else 0)

In [ ]:
df_train_es.head(2)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,labels
81812,es_0994681,product_es_0473680,reviewer_es_0370969,3,"Al poderse apilar, ordenas mucho mejor",Prácticas,es,furniture,1
8844,es_0419353,product_es_0907086,reviewer_es_0536262,1,No las e podido poner porque una de las luces ...,Mal,es,automotive,0


In [ ]:
df_train_es.tail(2)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,labels
146867,es_0950227,product_es_0177239,reviewer_es_0231213,4,Una buena cámara de gama media. Se nota una me...,SJCam SJ6 Legend,es,camera,1
121958,es_0371735,product_es_0679770,reviewer_es_0882536,4,Es como me lo imaginaba. Por el contrario de o...,Lo que estaba buscando,es,kitchen,1


#### 2.3) Adaptar formato del dataset 🔧

In [ ]:
# Convertir los DataFrames en objetos Dataset de la librería datasets
train_dataset = Dataset.from_pandas(df_train_es)
test_dataset = Dataset.from_pandas(df_test_es)
val_dataset = Dataset.from_pandas(df_val_es)

# Crear un DatasetDict con los conjuntos de datos
dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Ver la estructura
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels', '__index_level_0__'],
        num_rows: 147000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels', '__index_level_0__'],
        num_rows: 42000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels', '__index_level_0__'],
        num_rows: 21000
    })
})


### 3) Tokenización 📊

In [ ]:
model_checkpoint = "PlanTL-GOB-ES/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
def tokenize_reviews(examples):
    return tokenizer(examples["review_body"], truncation=True)

In [ ]:
columns = dataset["train"].column_names
columns.remove("labels")
encoded_dataset = dataset.map(tokenize_reviews, batched=True, remove_columns=columns)
print(encoded_dataset)

Map:   0%|          | 0/147000 [00:00<?, ? examples/s]

Map:   0%|          | 0/42000 [00:00<?, ? examples/s]

Map:   0%|          | 0/21000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 147000
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 42000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 21000
    })
})


### 4) Finetuning de la convnet 😨

In [ ]:
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Vamos a evaluar el accuracy como métrica de rendimiento**

In [ ]:
metric = evaluate.load("accuracy")
print(metric)

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return metric.compute(predictions=predictions, references=labels)

### 5) Hugging Face Hub 🤗

El modelo entrenado lo vamos a subir a Hugging Face Hub así lo podemos compartir con el mundo 😎

**Importante:** La nueva credencial que vamos a crear debe tener permisos de escritura (write).

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `demo-platzi-project` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authent

In [ ]:
!git config --global credential.helper store

### 6) Entrenamiento 💪

In [ ]:
model_name = model_checkpoint.split("/")[-1]

In [ ]:
print(model_name)

roberta-base-bne


In [ ]:
batch_size = 8
num_train_epochs=2
num_train_samples = 20_000
train_dataset = encoded_dataset["train"].shuffle(seed=42).select(range(num_train_samples))
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

In [ ]:
training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=True,
    hub_model_id=f"{model_name}-platzi-project-nlp-con-transformers"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["validation"],
    processing_class=tokenizer,
)

**Para este paso es necesario crearse una cuenta gratuita en [Weights & Biases](https://wandb.ai/home), porque el entrenamiento y las métricas se harán ahí.**

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cabustillo13 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.354500,0.348128,0.847167
2,0.262100,0.459290,0.857500


TrainOutput(global_step=5000, training_loss=0.3189806640625, metrics={'train_runtime': 1276.3661, 'train_samples_per_second': 31.339, 'train_steps_per_second': 3.917, 'total_flos': 1735126965673920.0, 'train_loss': 0.3189806640625, 'epoch': 2.0})

### 7) Guardar el modelo 💾

Para eso vamos a hacer un push a Hugging Face Hub

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/cabustillo13/roberta-base-bne-platzi-project-nlp-con-transformers/commit/e72bf9889b52fa1720b72b7ff9be8919030c363d', commit_message='End of training', commit_description='', oid='e72bf9889b52fa1720b72b7ff9be8919030c363d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cabustillo13/roberta-base-bne-platzi-project-nlp-con-transformers', endpoint='https://huggingface.co', repo_type='model', repo_id='cabustillo13/roberta-base-bne-platzi-project-nlp-con-transformers'), pr_revision=None, pr_num=None)

### 8) Hacer Predicciones en Producción 🤙

In [ ]:
from transformers import pipeline

Cargar el modelo una vez (al inicio de la aplicación)

In [ ]:
model_checkpoint = "cabustillo13/roberta-base-bne-platzi-project-nlp-con-transformers"
pipe = pipeline("sentiment-analysis", model=model_checkpoint)

config.json:   0%|          | 0.00/788 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

Device set to use cuda:0


Ejemplos de uso

In [ ]:
pipe("me encanto el pantalon!!!")

[{'label': 'LABEL_1', 'score': 0.9962030053138733}]

In [ ]:
pipe("Te obligan a comprar dos unidades")

[{'label': 'LABEL_0', 'score': 0.9433281421661377}]

In [ ]:
pipe("la peor compra de mi vida!!! no recomiendo!")

[{'label': 'LABEL_0', 'score': 0.9928492307662964}]